In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [2]:
file_path = 'Used_Cars.csv'
df = pd.read_csv(file_path)
df.head()

C:\Users\junio\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Unnamed: 0,id,vin,price,miles,stock_no,year,make,model,trim,...,drivetrain,transmission,fuel_type,engine_size,engine_block,seller_name,street,city,state,zip
0,0,38b2f52e-8f5d,1GCWGFCF3F1284719,20998.0,115879.0,W1T503168C,2015.0,Chevrolet,Express Cargo,Work Van,...,RWD,Automatic,E85 / Unleaded,4.8,V,nissan ellicott city,8569 Baltimore National Pike,Ellicott City,MD,21043
1,1,97ba4955-ccf0,WBY7Z8C59JVB87514,27921.0,7339.0,P33243,2018.0,BMW,i3,s,...,RWD,Automatic,Electric / Premium Unleaded,0.6,I,hendrick honda pompano beach,5381 N Federal Highway,Pompano Beach,FL,33064
2,2,be1da9fd-0f34,ML32F4FJ2JHF10325,11055.0,39798.0,WM2091A,2018.0,Mitsubishi,Mirage G4,SE,...,FWD,Automatic,Unleaded,1.2,I,russ darrow toyota,2700 West Washington St.,West Bend,WI,53095
3,3,84327e45-6cb6,1GCPTEE15K1291189,52997.0,28568.0,9U2Y425A,2019.0,Chevrolet,Colorado,ZR2,...,4WD,Automatic,Diesel,2.8,I,young kia,308 North Main Street,Layton,UT,84041
4,6,43847b9a-6fed,1B7HC16Y8YS543285,3995.0,137537.0,BP8246A,2000.0,Dodge,Ram Pickup,ST,...,RWD,Manual,Unleaded,5.2,V,baumann auto group,2379 W. State St.,Fremont,OH,43420


In [3]:
clean_df = df.drop(columns=['Unnamed: 0', 'id', 'vin', 'seller_name', 'street', 'stock_no', 'city', 'state', 'zip',])
clean_df.drop(clean_df.loc[clean_df['miles'] == 0].index, inplace=True)
clean_df = clean_df.sample(n=5000)
clean_df.head()

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,25999.0,65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
3193030,15988.0,106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
6139562,39975.0,48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,40000.0,17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,13900.0,152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V


In [4]:
clean_df.describe()

,price,miles,year,engine_size
count,5000.000000,5000.000000,5000.000000,5000.000000
mean,27481.732200,53015.980400,2016.614200,3.061400
std,16776.743306,42633.787332,3.381604,1.310654
min,1900.000000,2.000000,1997.000000,0.900000
25%,17000.000000,22969.500000,2015.000000,2.000000
50%,24487.000000,39411.500000,2018.000000,2.500000
75%,34836.250000,74728.500000,2019.000000,3.600000
max,334995.000000,518461.000000,2021.000000,8.100000


In [5]:
bins = [0, 5000, 10000, 15000, 25000, 50000, 100000, 400000]
labels = ['0-4,999', '5,000-9,999', '10,000-14999', '15,000-24,999', '25,000-49,999', '50,000-99,999', '100,000-400,000']

In [6]:
clean_df['price'] = pd.cut(clean_df['price'], bins, labels=labels )

In [7]:
clean_df

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V
...,...,...,...,...,...,...,...,...,...,...,...,...,...
119885,"15,000-24,999",41749.0,2014.0,Cadillac,ATS,Luxury Collection,Sedan,Car,4WD,Automatic,Premium Unleaded,2.0,I
2827731,"15,000-24,999",89455.0,2016.0,Chevrolet,Traverse,LTZ,SUV,Truck,4WD,Automatic,Unleaded,3.6,V
2550944,"25,000-49,999",35027.0,2018.0,Ford,Expedition,Limited,SUV,Truck,4WD,Automatic,Unleaded,3.5,V
3348921,"5,000-9,999",145515.0,2013.0,Hyundai,Sonata,SE,Sedan,Car,FWD,Automatic,Unleaded,2.0,I


In [8]:
clean_df.nunique()

price              7
miles           4840
year              25
make              45
model            451
trim             491
body_type         17
vehicle_type       2
drivetrain         3
transmission       2
fuel_type         15
engine_size       50
engine_block       3
dtype: int64

In [9]:
clean_df['price'].describe()

count              5000
unique                7
top       25,000-49,999
freq               2060
Name: price, dtype: object

In [10]:
drop_columns = ['price_0-4,999', 'price_5,000-9,999', 'price_10,000-14999', 'price_15,000-24,999', 'price_25,000-49,999', 'price_50,000-99,999', 'price_100,000-400,000']

In [11]:

X = pd.get_dummies(clean_df)
X = X.drop(columns=drop_columns)
y = clean_df['price']

In [12]:
X.describe()

,miles,year,engine_size,make_Acura,make_Alfa Romeo,make_Audi,make_BMW,make_Buick,make_Cadillac,make_Chevrolet,...,fuel_type_Electric / Unleaded,fuel_type_Premium Unleaded,fuel_type_Premium Unleaded / Unleaded,fuel_type_Premium Unleaded; Unleaded,fuel_type_Unleaded,fuel_type_Unleaded / E85,fuel_type_Unleaded / Unleaded,engine_block_H,engine_block_I,engine_block_V
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000
mean,53015.980400,2016.614200,3.061400,0.014400,0.002000,0.019000,0.040000,0.012400,0.012600,0.108400,...,0.016400,0.204800,0.000200,0.000800,0.672200,0.000400,0.00060,0.029600,0.518800,0.451600
std,42633.787332,3.381604,1.310654,0.119145,0.044681,0.136538,0.195979,0.110674,0.111551,0.310916,...,0.127021,0.403596,0.014142,0.028276,0.469458,0.019998,0.02449,0.169498,0.499696,0.497702
min,2.000000,1997.000000,0.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,22969.500000,2015.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
50%,39411.500000,2018.000000,2.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,1.000000,0.000000
75%,74728.500000,2019.000000,3.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.00000,0.000000,1.000000,1.000000
max,518461.000000,2021.000000,8.100000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000


In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(3750, 1032)

In [14]:
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.ensemble import RandomForestClassifier
clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)

In [15]:
# Display the confusion matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 10,   0,   0,   0,   1,   4,   0],
       [ 11,  90,   2,  18,   7,  24,   3],
       [  0,   0,   3,   0,   0,   0,   0],
       [  8, 107,   4, 227,  65,   9,  12],
       [  5,  11,  28,  54, 282,   3,  98],
       [ 29,  18,   1,   1,   4,  39,   0],
       [  0,   0,  14,   0,   9,   0,  49]], dtype=int64)

In [16]:
balanced_accuracy_score(y_test, y_pred)

0.6376459966046221

In [17]:
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

        0-4,999       0.16      0.67      0.96      0.26      0.80      0.62        15
   10,000-14999       0.40      0.58      0.88      0.47      0.71      0.49       155
100,000-400,000       0.06      1.00      0.96      0.11      0.98      0.96         3
  15,000-24,999       0.76      0.53      0.91      0.62      0.69      0.46       432
  25,000-49,999       0.77      0.59      0.89      0.66      0.72      0.50       481
    5,000-9,999       0.49      0.42      0.97      0.46      0.64      0.39        92
  50,000-99,999       0.30      0.68      0.90      0.42      0.78      0.60        72

    avg / total       0.66      0.56      0.90      0.59      0.71      0.49      1250



In [18]:
# List the features sorted in descending order by feature importance
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.09443787404730163, 'miles'),
 (0.08665315102997509, 'year'),
 (0.07047979387772273, 'engine_size'),
 (0.026709621452380136, 'fuel_type_Premium Unleaded'),
 (0.02333547328228797, 'fuel_type_Unleaded'),
 (0.021666795155132056, 'engine_block_V'),
 (0.0204386196179729, 'drivetrain_FWD'),
 (0.018015988720967754, 'engine_block_I'),
 (0.01759288131845611, 'body_type_Sedan'),
 (0.016943412686507595, 'drivetrain_4WD'),
 (0.01325253588879396, 'body_type_Pickup'),
 (0.012225623082130176, 'drivetrain_RWD'),
 (0.0117749224890198, 'make_Ford'),
 (0.011448814426911353, 'vehicle_type_Car'),
 (0.011148670790955077, 'vehicle_type_Truck'),
 (0.010893441306250317, 'body_type_SUV'),
 (0.010886725589550911, 'make_Chevrolet'),
 (0.00844673852992327, 'make_Nissan'),
 (0.007864140124658492, 'transmission_Automatic'),
 (0.007662599395937154, 'trim_SE'),
 (0.007556721593138199, 'trim_Limited'),
 (0.007455828227674203, 'make_Toyota'),
 (0.007423047450347455, 'make_Honda'),
 (0.006927091711439141, 'transmissio

# BRFC Focused on Make

In [19]:
clean_df.head()

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V


In [20]:
make_df = clean_df.drop(columns=['body_type', 'vehicle_type', 'drivetrain', 'fuel_type', 'engine_size', 'engine_block'])
make_df

,price,miles,year,make,model,trim,transmission
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,Automatic
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Automatic
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Automatic
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Automatic
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,Automatic
...,...,...,...,...,...,...,...
119885,"15,000-24,999",41749.0,2014.0,Cadillac,ATS,Luxury Collection,Automatic
2827731,"15,000-24,999",89455.0,2016.0,Chevrolet,Traverse,LTZ,Automatic
2550944,"25,000-49,999",35027.0,2018.0,Ford,Expedition,Limited,Automatic
3348921,"5,000-9,999",145515.0,2013.0,Hyundai,Sonata,SE,Automatic


In [28]:
X

,miles,year,"price_0-4,999","price_5,000-9,999","price_10,000-14999","price_15,000-24,999","price_25,000-49,999","price_50,000-99,999","price_100,000-400,000",make_Acura,...,trim_sDrive35i,trim_xDrive28i,trim_xDrive30i,trim_xDrive35d,trim_xDrive35i,trim_xDrive35i Premium,trim_xDrive40e,trim_xDrive50i,transmission_Automatic,transmission_Manual
5708709,65075.0,2015.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3193030,106320.0,2014.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6139562,48886.0,2016.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
6082007,17381.0,2017.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5930941,152169.0,2014.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119885,41749.0,2014.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2827731,89455.0,2016.0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2550944,35027.0,2018.0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3348921,145515.0,2013.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [30]:
X = pd.get_dummies(make_df)
X = X.drop(columns=["price_0-4,999", "price_5,000-9,999", "price_10,000-14999", "price_15,000-24,999", "price_25,000-49,999", "price_50,000-99,999", "price_100,000-400,000"])
y = make_df['price']

In [31]:
X.describe()

,miles,year,make_Acura,make_Alfa Romeo,make_Audi,make_BMW,make_Buick,make_Cadillac,make_Chevrolet,make_Chrysler,...,trim_sDrive35i,trim_xDrive28i,trim_xDrive30i,trim_xDrive35d,trim_xDrive35i,trim_xDrive35i Premium,trim_xDrive40e,trim_xDrive50i,transmission_Automatic,transmission_Manual
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,...,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,53015.980400,2016.614200,0.014400,0.002000,0.019000,0.040000,0.012400,0.012600,0.108400,0.010800,...,0.00060,0.001600,0.003600,0.000200,0.003600,0.000200,0.000800,0.000200,0.933800,0.066200
std,42633.787332,3.381604,0.119145,0.044681,0.136538,0.195979,0.110674,0.111551,0.310916,0.103371,...,0.02449,0.039972,0.059898,0.014142,0.059898,0.014142,0.028276,0.014142,0.248656,0.248656
min,2.000000,1997.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22969.500000,2015.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
50%,39411.500000,2018.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,74728.500000,2019.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,518461.000000,2021.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(3750, 991)

In [33]:
clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)

In [34]:
# Display the confusion matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 11,   0,   0,   0,   0,   4,   0],
       [ 13,  89,   1,  18,   6,  28,   0],
       [  0,   0,   1,   0,   0,   0,   2],
       [ 11,  97,  20, 152, 128,  14,  10],
       [  3,  26,  43,  28, 304,   4,  73],
       [ 28,  22,   0,   0,   1,  41,   0],
       [  0,   0,  19,   0,  17,   0,  36]], dtype=int64)

In [35]:
balanced_accuracy_score(y_test, y_pred)

0.5100544104050415

In [36]:
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

        0-4,999       0.17      0.73      0.96      0.27      0.84      0.69        15
   10,000-14999       0.38      0.57      0.87      0.46      0.71      0.48       155
100,000-400,000       0.01      0.33      0.93      0.02      0.56      0.29         3
  15,000-24,999       0.77      0.35      0.94      0.48      0.58      0.31       432
  25,000-49,999       0.67      0.63      0.80      0.65      0.71      0.50       481
    5,000-9,999       0.45      0.45      0.96      0.45      0.65      0.40        92
  50,000-99,999       0.30      0.50      0.93      0.37      0.68      0.44        72

    avg / total       0.62      0.51      0.88      0.53      0.66      0.42      1250



In [37]:
# List the features sorted in descending order by feature importance
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.1255239489515081, 'miles'),
 (0.11200616391666482, 'year'),
 (0.014437682958624541, 'make_Toyota'),
 (0.013548249262345666, 'make_Chevrolet'),
 (0.013374041604819196, 'trim_SE'),
 (0.012679346957228927, 'make_Ford'),
 (0.012169122896392914, 'trim_Base'),
 (0.011465463942754838, 'transmission_Manual'),
 (0.010530324941325385, 'transmission_Automatic'),
 (0.008930201019616701, 'make_Honda'),
 (0.00890600858243586, 'model_Corvette'),
 (0.00869203568910952, 'trim_Limited'),
 (0.00820827117290635, 'make_Hyundai'),
 (0.008168961569668757, 'trim_LT'),
 (0.007952183474890039, 'make_Porsche'),
 (0.0079171157674536, 'trim_S'),
 (0.00783698748053368, 'make_Nissan'),
 (0.007708643271550752, 'make_GMC'),
 (0.0075071623347392966, 'trim_LX'),
 (0.0074222135604671904, 'make_Jeep'),
 (0.007171500111435825, 'make_RAM'),
 (0.007118043798513709, 'make_Audi'),
 (0.006995455155322679, 'make_Kia'),
 (0.006932194826848821, 'make_Cadillac'),
 (0.006466471102480245, 'model_Corolla'),
 (0.006315552257058931,

# Luxury Makes

In [38]:
clean_df.head()

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V


In [39]:
clean_df['make'].unique()

array(['Lexus', 'Mercedes-Benz', 'Ford', 'Dodge', 'Jeep', 'Volkswagen',
       'RAM', 'INFINITI', 'Hyundai', 'Kia', 'Honda', 'Land Rover',
       'Chevrolet', 'Cadillac', 'GMC', 'Audi', 'Chrysler', 'Subaru',
       'BMW', 'Volvo', 'Toyota', 'Mazda', 'Nissan', 'GENESIS', 'Acura',
       'Hummer', 'Porsche', 'Lincoln', 'Buick', 'Jaguar', 'Scion',
       'Alfa Romeo', 'FIAT', 'Mitsubishi', 'MINI', 'Lamborghini',
       'Saturn', 'Pontiac', 'Maserati', 'smart', 'McLaren', 'Rolls-Royce',
       'Ferrari', 'Suzuki', 'Mercury'], dtype=object)

In [41]:
luxury_makes = ['Mercedes-Benz', 'Cadillac', 'BMW', 'Jaguar', 'Porsche', 'Maserati', 'Bentley', 'Rolls-Royce', 'Aston Martin', 'Ferrari', 'Lamborghini', 'Alfa Romeo',]

In [54]:
luxury_df = clean_df.loc[clean_df['make'].isin(luxury_makes)]

In [55]:
luxury_df

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
1503202,"15,000-24,999",120622.0,2012.0,Cadillac,Escalade,Premium,SUV,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
3366787,"50,000-99,999",14737.0,2020.0,Mercedes-Benz,E-Class,E350,Sedan,Car,RWD,Automatic,Premium Unleaded,2.0,I
3442587,"25,000-49,999",36018.0,2016.0,BMW,4 Series,435i,Coupe,Car,RWD,Automatic,Premium Unleaded,3.0,I
4822612,"25,000-49,999",38025.0,2018.0,Mercedes-Benz,C-Class,C300,Sedan,Car,4WD,Automatic,Premium Unleaded,2.0,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3459348,"25,000-49,999",34389.0,2017.0,BMW,5 Series,540i,Sedan,Car,4WD,Automatic,Premium Unleaded,3.0,I
1868684,"50,000-99,999",26355.0,2020.0,Jaguar,F-Pace,S,SUV,Truck,4WD,Automatic,Premium Unleaded,3.0,V
4746049,"25,000-49,999",7086.0,2020.0,BMW,X2,28i,SUV,Truck,4WD,Automatic,Premium Unleaded,2.0,I
4996514,"50,000-99,999",4905.0,2020.0,Mercedes-Benz,E-Class,AMGA E53,Coupe,Car,4WD,Automatic,Electric / Premium Unleaded,3.0,I


In [56]:
make_df = luxury_df.drop(columns=['body_type', 'vehicle_type', 'drivetrain', 'fuel_type', 'engine_size', 'engine_block'])
make_df

,price,miles,year,make,model,trim,transmission
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Automatic
1503202,"15,000-24,999",120622.0,2012.0,Cadillac,Escalade,Premium,Automatic
3366787,"50,000-99,999",14737.0,2020.0,Mercedes-Benz,E-Class,E350,Automatic
3442587,"25,000-49,999",36018.0,2016.0,BMW,4 Series,435i,Automatic
4822612,"25,000-49,999",38025.0,2018.0,Mercedes-Benz,C-Class,C300,Automatic
...,...,...,...,...,...,...,...
3459348,"25,000-49,999",34389.0,2017.0,BMW,5 Series,540i,Automatic
1868684,"50,000-99,999",26355.0,2020.0,Jaguar,F-Pace,S,Automatic
4746049,"25,000-49,999",7086.0,2020.0,BMW,X2,28i,Automatic
4996514,"50,000-99,999",4905.0,2020.0,Mercedes-Benz,E-Class,AMGA E53,Automatic


In [57]:
X = pd.get_dummies(make_df)
X = X.drop(columns=["price_0-4,999", "price_5,000-9,999", "price_10,000-14999", "price_15,000-24,999", "price_25,000-49,999", "price_50,000-99,999", "price_100,000-400,000"])
y = make_df['price']

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(358, 220)

In [59]:
clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)

In [60]:
# Display the confusion matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 0,  0,  0,  0,  0,  0,  0],
       [ 3,  0,  0,  3,  0,  1,  0],
       [ 0,  0,  1,  0,  0,  0,  1],
       [ 2,  0,  0, 10,  2,  1,  0],
       [ 3,  0,  2, 18, 29,  0, 25],
       [ 0,  1,  0,  0,  0,  0,  0],
       [ 1,  0,  1,  2,  6,  0,  8]], dtype=int64)

In [61]:
balanced_accuracy_score(y_test, y_pred)

C:\Users\junio\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1987: UserWarning: y_pred contains classes not in y_true
  warnings.warn("y_pred contains classes not in y_true")


0.33128908128908124

In [62]:
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

        0-4,999       0.00      0.00      0.93      0.00      0.00      0.00         0
   10,000-14999       0.00      0.00      0.99      0.00      0.00      0.00         7
100,000-400,000       0.25      0.50      0.97      0.33      0.70      0.46         2
  15,000-24,999       0.30      0.67      0.78      0.42      0.72      0.51        15
  25,000-49,999       0.78      0.38      0.81      0.51      0.55      0.29        77
    5,000-9,999       0.00      0.00      0.98      0.00      0.00      0.00         1
  50,000-99,999       0.24      0.44      0.75      0.31      0.58      0.32        18

    avg / total       0.58      0.40      0.81      0.43      0.54      0.31       120



C:\Users\junio\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\junio\anaconda3\envs\mlenv\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Sensitivity is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [63]:
# List the features sorted in descending order by feature importance
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.12752564212819628, 'miles'),
 (0.10984595803233574, 'year'),
 (0.06812822497420021, 'make_Cadillac'),
 (0.05088249054007568, 'make_BMW'),
 (0.05042710124985668, 'trim_Base'),
 (0.04020726694186447, 'model_STS'),
 (0.03392368421052632, 'transmission_Automatic'),
 (0.03006944444444444, 'make_Mercedes-Benz'),
 (0.02740118679050568, 'model_X5'),
 (0.016989766081871343, 'transmission_Manual'),
 (0.014752751977984176, 'model_Escalade ESV'),
 (0.013249226006191949, 'make_Porsche'),
 (0.012885526315789472, 'model_E-Class'),
 (0.012521499828001374, 'model_CLA-Class'),
 (0.012433551198257085, 'model_Panamera'),
 (0.011912280701754388, 'trim_C300 Sport'),
 (0.010957172342621259, 'model_AMG GT Roadster'),
 (0.010938510491916065, 'trim_Luxury Collection'),
 (0.010624183006535946, 'trim_R'),
 (0.010196307762871231, 'make_Ferrari'),
 (0.009967592592592594, 'trim_GLE350'),
 (0.00980108359133127, 'trim_Premium Luxury'),
 (0.009759803921568628, 'model_5 Series'),
 (0.009184640522875818, 'trim_Luxury

# Non-Luxury Makes

In [64]:
clean_df.head()

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
3193030,"15,000-24,999",106320.0,2014.0,Mercedes-Benz,CLA-Class,CLA250,Coupe,Car,FWD,Automatic,Premium Unleaded,2.0,I
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V


In [65]:
clean_df['make'].unique()

array(['Lexus', 'Mercedes-Benz', 'Ford', 'Dodge', 'Jeep', 'Volkswagen',
       'RAM', 'INFINITI', 'Hyundai', 'Kia', 'Honda', 'Land Rover',
       'Chevrolet', 'Cadillac', 'GMC', 'Audi', 'Chrysler', 'Subaru',
       'BMW', 'Volvo', 'Toyota', 'Mazda', 'Nissan', 'GENESIS', 'Acura',
       'Hummer', 'Porsche', 'Lincoln', 'Buick', 'Jaguar', 'Scion',
       'Alfa Romeo', 'FIAT', 'Mitsubishi', 'MINI', 'Lamborghini',
       'Saturn', 'Pontiac', 'Maserati', 'smart', 'McLaren', 'Rolls-Royce',
       'Ferrari', 'Suzuki', 'Mercury'], dtype=object)

In [66]:
nonlux_df = clean_df.loc[clean_df['make'].isin(luxury_makes) == False]

In [67]:
nonlux_df

,price,miles,year,make,model,trim,body_type,vehicle_type,drivetrain,transmission,fuel_type,engine_size,engine_block
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,SUV,Truck,FWD,Automatic,Premium Unleaded,3.5,V
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Pickup,Truck,4WD,Automatic,E85 / Unleaded,6.2,V
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Sedan,Car,RWD,Automatic,Unleaded,5.7,V
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,SUV,Truck,RWD,Automatic,E85 / Unleaded,3.6,V
59712,"10,000-14999",56636.0,2014.0,Volkswagen,GTI,Wolfsburg Edition,Hatchback,Car,FWD,Manual,Premium Unleaded,2.0,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...
508259,"25,000-49,999",26079.0,2018.0,Honda,Accord,EX-L,Sedan,Car,FWD,Automatic,Unleaded,1.5,I
2827731,"15,000-24,999",89455.0,2016.0,Chevrolet,Traverse,LTZ,SUV,Truck,4WD,Automatic,Unleaded,3.6,V
2550944,"25,000-49,999",35027.0,2018.0,Ford,Expedition,Limited,SUV,Truck,4WD,Automatic,Unleaded,3.5,V
3348921,"5,000-9,999",145515.0,2013.0,Hyundai,Sonata,SE,Sedan,Car,FWD,Automatic,Unleaded,2.0,I


In [68]:
make_df = nonlux_df.drop(columns=['body_type', 'vehicle_type', 'drivetrain', 'fuel_type', 'engine_size', 'engine_block'])
make_df

,price,miles,year,make,model,trim,transmission
5708709,"25,000-49,999",65075.0,2015.0,Lexus,RX,350,Automatic
6139562,"25,000-49,999",48886.0,2016.0,Ford,F-250 Super Duty,XLT,Automatic
6082007,"25,000-49,999",17381.0,2017.0,Dodge,Charger,Daytona,Automatic
5930941,"10,000-14999",152169.0,2014.0,Jeep,Grand Cherokee,Limited,Automatic
59712,"10,000-14999",56636.0,2014.0,Volkswagen,GTI,Wolfsburg Edition,Manual
...,...,...,...,...,...,...,...
508259,"25,000-49,999",26079.0,2018.0,Honda,Accord,EX-L,Automatic
2827731,"15,000-24,999",89455.0,2016.0,Chevrolet,Traverse,LTZ,Automatic
2550944,"25,000-49,999",35027.0,2018.0,Ford,Expedition,Limited,Automatic
3348921,"5,000-9,999",145515.0,2013.0,Hyundai,Sonata,SE,Automatic


In [69]:
X = pd.get_dummies(make_df)
X = X.drop(columns=["price_0-4,999", "price_5,000-9,999", "price_10,000-14999", "price_15,000-24,999", "price_25,000-49,999", "price_50,000-99,999", "price_100,000-400,000"])
y = make_df['price']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1)
X_train.shape

(3391, 784)

In [71]:
clf = BalancedRandomForestClassifier(random_state=1, n_estimators=100).fit(X_train, y_train)

In [72]:
# Display the confusion matrix
y_pred = clf.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[ 10,   0,   0,   0,   0,   3,   0],
       [  5,  64,   7,  21,   3,  37,   2],
       [  0,   0,   0,   0,   0,   0,   1],
       [  8,  73,  27, 149,  78,  24,  52],
       [  5,  18,  42,  49, 156,   6, 163],
       [ 31,  14,   0,   0,   0,  44,   0],
       [  1,   0,   6,   1,   4,   0,  27]], dtype=int64)

In [73]:
balanced_accuracy_score(y_test, y_pred)

0.447747946783254

In [74]:
print(classification_report_imbalanced(y_test, y_pred))

                       pre       rec       spe        f1       geo       iba       sup

        0-4,999       0.17      0.77      0.96      0.27      0.86      0.72        13
   10,000-14999       0.38      0.46      0.89      0.42      0.64      0.39       139
100,000-400,000       0.00      0.00      0.93      0.00      0.00      0.00         1
  15,000-24,999       0.68      0.36      0.90      0.47      0.57      0.31       411
  25,000-49,999       0.65      0.36      0.88      0.46      0.56      0.30       439
    5,000-9,999       0.39      0.49      0.93      0.43      0.68      0.44        89
  50,000-99,999       0.11      0.69      0.80      0.19      0.74      0.55        39

    avg / total       0.58      0.40      0.89      0.44      0.59      0.34      1131



In [75]:
# List the features sorted in descending order by feature importance
sorted(zip(clf.feature_importances_, X.columns), reverse=True)

[(0.10986299134551329, 'year'),
 (0.10389147739644139, 'miles'),
 (0.020792016552693027, 'make_Chevrolet'),
 (0.020520582623081002, 'model_Corvette'),
 (0.017542484361362125, 'trim_Base'),
 (0.016149305479074706, 'make_Ford'),
 (0.014750668826016031, 'trim_SE'),
 (0.013577464726505933, 'make_Toyota'),
 (0.013551944678911102, 'transmission_Automatic'),
 (0.012849109649971551, 'make_Audi'),
 (0.012519393246875683, 'transmission_Manual'),
 (0.012368275496339988, 'trim_Limited'),
 (0.01174280526364208, 'make_Honda'),
 (0.011688744579012948, 'trim_S'),
 (0.010969961584230052, 'make_Hyundai'),
 (0.010433025396353485, 'make_Jeep'),
 (0.010062130926657557, 'make_Kia'),
 (0.010011758802788656, 'make_RAM'),
 (0.00980879586120273, 'trim_SLT'),
 (0.009548849477717858, 'trim_1LT'),
 (0.009366904769690957, 'trim_SVR'),
 (0.009273075559726746, 'make_GMC'),
 (0.008983360612017606, 'model_Ram 1500 Pickup'),
 (0.008982827677121068, 'trim_LX'),
 (0.008529976186129993, 'make_Nissan'),
 (0.0079281399711198